In [1]:
%pylab inline 


Populating the interactive namespace from numpy and matplotlib


In [2]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding
from collections import Counter
import time
import progressbar as pb
from tqdm import tqdm
import os
import shutil
from empyrical import sortino_ratio, calmar_ratio, omega_ratio
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, ActorCriticPolicy, FeedForwardPolicy
from stable_baselines.common.vec_env import SubprocVecEnv, VecEnv, VecEnvWrapper
from stable_baselines import A2C, PPO2
from stable_baselines.common.vec_env import DummyVecEnv
import tensorflow as tf
from trading_env import TradingEnv

In [3]:
# load the market data
input_source = np.load(open('data_dummy.npy','rb'))
to_predict = np.load(open('data_dummy_targets.npy','rb'))

to_predict = to_predict[3,:].reshape(-1)

input_source = input_source.T

In [4]:
is_orig = np.copy(input_source)
cp = int(0.8*len(input_source))
test_input_source = input_source[cp:, :]
test_to_predict = to_predict[cp:]
input_source = input_source[0:cp, :]
to_predict = to_predict[0:cp]

In [5]:
bars_per_episode = 1000
winlen = 10
traded_amt = 1
initial_balance = 10000
commission = 0
slippage = 0.0

In [7]:
# multiprocess environment
#n_cpu = 16
#env = SubprocVecEnv([lambda: TradingEnv(input_source, to_predict,
#                                         winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt, initial_balance=initial_balance,
#                                         commission=commission, slippage=slippage) for i in range(n_cpu)])
env = TradingEnv(input_source, to_predict,
                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt, initial_balance=initial_balance,
                 commission=commission, slippage=slippage
                 )
env = DummyVecEnv([lambda: env])

t = 0
# Custom MLP policy of two layers of size 32 each with tanh activation function
policy_kwargs = dict(act_fun=tf.nn.relu, net_arch=[64, 64])

#[shutil.rmtree('/home/peter/tblog/'+x) for x in os.listdir('/home/peter/tblog/') if x]
model = PPO2(MlpPolicy, env, n_steps=64, verbose=1, nminibatches=1, policy_kwargs=policy_kwargs, 
             gamma=0.8,#0.99,
             ent_coef=0.05,#0.01,
             learning_rate=0.00025,
             vf_coef=0.5,
             max_grad_norm=0.5,
             lam=0.95,
             tensorboard_log='/home/peter/tblog')
try:
    model.learn(total_timesteps=10_000_000)
    model.save("ppo2_trading")
except KeyboardInterrupt:
    pass

W0706 10:41:02.009387 139788412827456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/stable_baselines/common/tf_util.py:98: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0706 10:41:02.010814 139788412827456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/stable_baselines/common/tf_util.py:107: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0706 10:41:02.046146 139788412827456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/stable_baselines/common/policies.py:114: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0706 10:41:02.047315 139788412827456 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/stable_baselines/common/input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 10:41:02.049931 139788412827456 deprecation.py:323] From /u

--------------------------------------
| approxkl           | 2.3226072e-05 |
| clipfrac           | 0.0           |
| explained_variance | -0.00816      |
| fps                | 270           |
| n_updates          | 1             |
| policy_entropy     | 1.0985864     |
| policy_loss        | -0.001956232  |
| serial_timesteps   | 64            |
| time_elapsed       | 2.15e-06      |
| total_timesteps    | 64            |
| value_loss         | 481.517       |
--------------------------------------
---------------------------------------
| approxkl           | 1.7824128e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0.00628        |
| fps                | 1613           |
| n_updates          | 2              |
| policy_entropy     | 1.0984509      |
| policy_loss        | -0.00072117243 |
| serial_timesteps   | 128            |
| time_elapsed       | 0.238          |
| total_timesteps    | 128            |
| value_loss         | 424.76416      |
-------------

---------------------------------------
| approxkl           | 2.0490463e-05  |
| clipfrac           | 0.0            |
| explained_variance | 0.0289         |
| fps                | 1475           |
| n_updates          | 18             |
| policy_entropy     | 1.0966382      |
| policy_loss        | -0.00025416142 |
| serial_timesteps   | 1152           |
| time_elapsed       | 0.835          |
| total_timesteps    | 1152           |
| value_loss         | 525.5556       |
---------------------------------------
--------------------------------------
| approxkl           | 2.5473453e-05 |
| clipfrac           | 0.0           |
| explained_variance | 0.0493        |
| fps                | 1695          |
| n_updates          | 19            |
| policy_entropy     | 1.0965564     |
| policy_loss        | -0.0025573361 |
| serial_timesteps   | 1216          |
| time_elapsed       | 0.879         |
| total_timesteps    | 1216          |
| value_loss         | 829.5355      |
------------

---------------------------------------
| approxkl           | 0.000119199976 |
| clipfrac           | 0.0            |
| explained_variance | 0.0749         |
| fps                | 2086           |
| n_updates          | 35             |
| policy_entropy     | 1.0112743      |
| policy_loss        | -0.0044248756  |
| serial_timesteps   | 2240           |
| time_elapsed       | 1.46           |
| total_timesteps    | 2240           |
| value_loss         | 6151.208       |
---------------------------------------
--------------------------------------
| approxkl           | 5.989291e-05  |
| clipfrac           | 0.0           |
| explained_variance | 0.0812        |
| fps                | 1904          |
| n_updates          | 36            |
| policy_entropy     | 1.0683951     |
| policy_loss        | -0.0017958728 |
| serial_timesteps   | 2304          |
| time_elapsed       | 1.49          |
| total_timesteps    | 2304          |
| value_loss         | 353.67487     |
------------

--------------------------------------
| approxkl           | 0.00018991264 |
| clipfrac           | 0.0           |
| explained_variance | 0.181         |
| fps                | 1819          |
| n_updates          | 52            |
| policy_entropy     | 0.87595046    |
| policy_loss        | 0.0015488509  |
| serial_timesteps   | 3328          |
| time_elapsed       | 2.04          |
| total_timesteps    | 3328          |
| value_loss         | 16539.611     |
--------------------------------------
--------------------------------------
| approxkl           | 9.3521485e-05 |
| clipfrac           | 0.0           |
| explained_variance | 0.0504        |
| fps                | 1754          |
| n_updates          | 53            |
| policy_entropy     | 0.92641044    |
| policy_loss        | -0.0022295753 |
| serial_timesteps   | 3392          |
| time_elapsed       | 2.07          |
| total_timesteps    | 3392          |
| value_loss         | 7479.717      |
-------------------------

...

--------------------------------------
| approxkl           | 1.5411151e-08 |
| clipfrac           | 0.0           |
| explained_variance | 0.999         |
| fps                | 1802          |
| n_updates          | 1520          |
| policy_entropy     | 0.00239945    |
| policy_loss        | -4.662201e-06 |
| serial_timesteps   | 97280         |
| time_elapsed       | 56.2          |
| total_timesteps    | 97280         |
| value_loss         | 31.246498     |
--------------------------------------
--------------------------------------
| approxkl           | 7.8616125e-10 |
| clipfrac           | 0.0           |
| explained_variance | 0.999         |
| fps                | 1738          |
| n_updates          | 1521          |
| policy_entropy     | 0.00077313976 |
| policy_loss        | 1.2563542e-06 |
| serial_timesteps   | 97344         |
| time_elapsed       | 56.3          |
| total_timesteps    | 97344         |
| value_loss         | 20.443098     |
-------------------------

-------------------------------------
| approxkl           | 0.0023547723 |
| clipfrac           | 0.03125      |
| explained_variance | 0.998        |
| fps                | 1614         |
| n_updates          | 1536         |
| policy_entropy     | 0.06878638   |
| policy_loss        | 0.0027661826 |
| serial_timesteps   | 98304        |
| time_elapsed       | 56.9         |
| total_timesteps    | 98304        |
| value_loss         | 17.150837    |
-------------------------------------
-------------------------------------
| approxkl           | 0.0005810375 |
| clipfrac           | 0.01171875   |
| explained_variance | 0.999        |
| fps                | 1284         |
| n_updates          | 1537         |
| policy_entropy     | 0.040890686  |
| policy_loss        | 0.008900664  |
| serial_timesteps   | 98368        |
| time_elapsed       | 56.9         |
| total_timesteps    | 98368        |
| value_loss         | 14.664043    |
-------------------------------------
------------

--------------------------------------
| approxkl           | 0.00071462285 |
| clipfrac           | 0.0078125     |
| explained_variance | 0.999         |
| fps                | 1649          |
| n_updates          | 1553          |
| policy_entropy     | 0.06817927    |
| policy_loss        | -0.0005390127 |
| serial_timesteps   | 99392         |
| time_elapsed       | 57.6          |
| total_timesteps    | 99392         |
| value_loss         | 7.747581      |
--------------------------------------
--------------------------------------
| approxkl           | 0.00016495553 |
| clipfrac           | 0.00390625    |
| explained_variance | 0.999         |
| fps                | 1860          |
| n_updates          | 1554          |
| policy_entropy     | 0.03011183    |
| policy_loss        | -0.0035648476 |
| serial_timesteps   | 99456         |
| time_elapsed       | 57.6          |
| total_timesteps    | 99456         |
| value_loss         | 5.849383      |
-------------------------

--------------------------------------
| approxkl           | 2.0076257e-05 |
| clipfrac           | 0.0           |
| explained_variance | 0.999         |
| fps                | 1653          |
| n_updates          | 1570          |
| policy_entropy     | 0.039769016   |
| policy_loss        | -8.117408e-06 |
| serial_timesteps   | 100480        |
| time_elapsed       | 58.3          |
| total_timesteps    | 100480        |
| value_loss         | 6.2182903     |
--------------------------------------
-------------------------------------
| approxkl           | 9.536883e-07 |
| clipfrac           | 0.0          |
| explained_variance | 0.999        |
| fps                | 1483         |
| n_updates          | 1571         |
| policy_entropy     | 0.02035758   |
| policy_loss        | 9.097392e-05 |
| serial_timesteps   | 100544       |
| time_elapsed       | 58.3         |
| total_timesteps    | 100544       |
| value_loss         | 7.585747     |
-------------------------------------

--------------------------------------
| approxkl           | 1.2124087e-08 |
| clipfrac           | 0.0           |
| explained_variance | 0.999         |
| fps                | 1702          |
| n_updates          | 1587          |
| policy_entropy     | 0.005273466   |
| policy_loss        | 1.0072719e-05 |
| serial_timesteps   | 101568        |
| time_elapsed       | 59            |
| total_timesteps    | 101568        |
| value_loss         | 28.85897      |
--------------------------------------


In [ ]:
env = TradingEnv(test_input_source, test_to_predict,
                 winlen=winlen, bars_per_episode=bars_per_episode, traded_amt=traded_amt,
                 commission=commission, slippage=slippage
                 )
env = DummyVecEnv([lambda: env])

# visualize the behavior for one random episode
bars_per_episode = 1000

nstate = model.initial_state  # get the initial state vector for the reccurent network
#dones = np.zeros(nstate.shape[0])  # set all environment to not done
nstate=None

observation = env.envs[0].reset()#env.reset()
done = False
navs = []
acts = []
for i in tqdm(range(bars_per_episode)):
    action, nstate = model.predict([observation], state=nstate)
    acts.append(action)
    observation, reward, done, info = env.envs[0].step(action)#env.step(action)
    if done:
        break
    navs.append(env.get_attr('balance')[0])


kl = []
t = 0
for n in np.diff(np.vstack(navs).reshape(-1)):
    t = t + n
    kl.append(t)
plot(kl);

In [ ]:
# calculate the likelihood of success for any given episode
try:
    l = 2500

    krl = []
    p = pb.ProgressBar(max_value=l)
    for i in range(l):
        p.update(i)
        observation = env.envs[0].reset()
        done = False
        navs = []
        for i in (range(bars_per_episode)):
            action, nstate = model.predict([observation], state=nstate)
            acts.append(action)
            observation, reward, done, info = env.envs[0].step(action)#env.step(action)
            navs.append(env.get_attr('balance')[0])
        krl.append(sum(navs))
    p.finish()
except KeyboardInterrupt:
    pass

krl = np.array(krl)
print('Profit likelihood: %3.3f%%' % (100*(sum(krl > 0) / len(krl))))

In [ ]:
hist(krl,20);